In [ ]:
# Mount Google Drive to access shared datasets
from google.colab import drive
drive.mount('/content/drive')

import os
import tensorflow as tf

# Base of your shared drive
ROOT_DIR    = '/content/drive/Shareddrives/OmniClick Team'
DATA_DIR    = os.path.join(ROOT_DIR, 'datasets')
UNITY_DIR   = os.path.join(DATA_DIR, 'UnityEyes')    # <> where train/val/test live

# Hyperparameters
IMG_SIZE   = (96, 96)   # all images will be resized to 96×96
BATCH_SIZE = 32
SEED       = 42028
AUTOTUNE   = tf.data.AUTOTUNE
EPOCHS     = 100        # we’ll rely on EarlyStopping to halt earlier


In [ ]:
# 1) Load folder-structured datasets
unity_train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(UNITY_DIR, 'train'),
    labels='inferred', label_mode='categorical',
    image_size=IMG_SIZE, batch_size=BATCH_SIZE,
    shuffle=True, seed=SEED
)
unity_val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(UNITY_DIR, 'val'),
    labels='inferred', label_mode='categorical',
    image_size=IMG_SIZE, batch_size=BATCH_SIZE,
    shuffle=False
)
unity_test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(UNITY_DIR, 'test'),
    labels='inferred', label_mode='categorical',
    image_size=IMG_SIZE, batch_size=BATCH_SIZE,
    shuffle=False
)

# 2) Normalize pixels and prepare pipeline
def preprocess(image, label):
    """Scale image pixels to [0,1]."""
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

unity_train_ds = (
    unity_train_ds
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .cache()
    .shuffle(1000, seed=SEED)
    .prefetch(AUTOTUNE)
)
unity_val_ds = (
    unity_val_ds
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .cache()
    .prefetch(AUTOTUNE)
)
unity_test_ds = (
    unity_test_ds
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .cache()
    .prefetch(AUTOTUNE)
)

# 3) Determine number of Unity class labels
NUM_CLASSES_UNITY = len(unity_train_ds.class_names)
print("UnityEyes classes:", unity_train_ds.class_names)


In [ ]:
from tensorflow.keras import layers, models, Input

def build_baseline(num_classes):
    """
    Build a simple custom CNN:
    Conv(32)→MaxPool→Conv(64)→MaxPool→Conv(128)→GAP→Dense(128)→Dropout(0.3)→Softmax
    """
    return models.Sequential([
        Input(shape=(*IMG_SIZE, 3)),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation='relu'),
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])

# Instantiate the model for Unity pretraining
model = build_baseline(NUM_CLASSES_UNITY)
model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Compile with Adam optimizer and categorical crossentropy
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# EarlyStopping and LR reduction on plateau
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        verbose=1
    )
]


In [ ]:
# Train up to EPOCHS=100, but stop early when val_loss plateaus
history_unity = model.fit(
    unity_train_ds,
    validation_data=unity_val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)


In [ ]:
# 1) Save weights for later fine-tuning
WEIGHTS_PATH = os.path.join(ROOT_DIR, 'notebooks/weights/unity_pretrained.h5')
model.save_weights(WEIGHTS_PATH)
print("Saved Unity pretraining weights to:", WEIGHTS_PATH)

# 2) Evaluate on Unity test set (optional)
loss_u, acc_u = model.evaluate(unity_test_ds)
print(f"Unity Test Loss: {loss_u:.4f}, Accuracy: {acc_u:.4f}")
